# Getting all available Swiss petrol stations using the overpass Web API

## Libraries and settings

In [ ]:
# Libraries
import os
import json
import folium
import requests
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

## Overpass turbo query to get available petrol stations

In [ ]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["amenity"="fuel"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to .json file
with open('petrol_stations.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = pd.json_normalize(data)
df.head()

## Save data frame to a .csv file

In [ ]:
# Write data frame to .csv - file
df.to_csv('petrol-stations.csv', index=False, sep=";", encoding='utf-8')

## Use a pivot table to count the number of petrol stations per operator

In [ ]:
# Create pivot table
tab = pd.pivot_table(df[['id', 'tags.name']], 
                       values=['id'], 
                       index=['tags.name'],
                       aggfunc='count')

# Sort pivot table
tab_sorted = tab.sort_values(by=['id'], ascending=False)
tab_sorted

## Plot petrol stations on map

In [ ]:
# Subset of petrol stations by brand
locations = df[["lat", 
                "lon", 
                "tags.brand", 
                "tags.shop"]].loc[df["tags.brand"].isin(['Migrol', 
                                                         'Agip', 
                                                         'Shell'])]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                   popup=location_info["tags.brand"]).add_to(map)

# Plot map
map

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')